In [ ]:
import logging
from simd_r_drive_ws_client import DataStoreWsClient
import simd_r_drive_ws_client

simd_r_drive_ws_client.setup_logging(logging.info)

In [ ]:
client = DataStoreWsClient("127.0.0.1:53645")
# client = DataStoreWsClient("192.168.0.10:3000")

In [ ]:
from simd_r_drive_ws_client import DataStoreWsClient
import os, secrets, time, sys



# ── 1.  Prepare a *big* payload  ─────────────────────────────────────────
#
# • 200 000 keys, each key ≈ 10 bytes  →  ~2 MB of key-names
# • 64-byte value per key              →  ~12 MB of payload
#   (big enough to burst an 8 or 16 MiB frame cap)

KEYS     = [f"key-{i}".encode() for i in range(200_000)]
PAYLOAD  = secrets.token_bytes(64)

print(f"Writing {len(KEYS)} keys …")
start = time.perf_counter()

# for k in KEYS:
#     c.write(k, PAYLOAD)           # tiny individual writes = no problem
# client.batch_write(list(zip(KEYS, [PAYLOAD] * len(KEYS))))
# print(f"✓ wrote in {time.perf_counter()-start:.1f}s")

# ── 2.  Add one missing key so we get a None in the output  ──────────────
batch_query = KEYS + [b"missing-key"]

print("\nIssuing giant batch_read … (this is expected to blow up)")
try:
    _ = client.batch_read(batch_query)     #  ← BOOM (if frame too large)
    print("❗  Surprisingly succeeded – your frame cap is already high.")
except Exception as e:
    print("💥  batch_read raised:", e)
    print("Most likely the server closed the frame early → decoder saw an "
          "invalid tag byte and bailed out.")


In [ ]:
# client.batch_write([
#     (b"testing1221", b"hello there!!!!"),
#     (b"testing1", b"12345"),
#     (b"testing2", b"12345"),
#     (b"bytes", bytes(1024))
# ])

client.stage_write(b"hello", b"world")


# print("reading")
# for i in range(1, 1000):
#     print(i)
#     client.write(i.to_bytes(4), i.to_bytes(4))

#     client.read(i.to_bytes(4))

# display(client.read(b"__triplet_count__"))

In [ ]:
client.stage_write_flush()

In [ ]:
# len(client.read(b"bytes"))
client.read(b"hello")